<a href="https://colab.research.google.com/github/Bahramif/Deep-Trading/blob/master/Weekly_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()

Saving My First Project-071ea935f8a8.json to My First Project-071ea935f8a8.json


In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My First Project-071ea935f8a8.json"
!pip install --upgrade google-cloud-vision
!pip install requests
def grocery_list():
  shopping_list=['steak','sirloin','ham','blackberry','cherry','grapefruit','mango','kiwi']
  Meat=['salmon','trout','tilapia','ground beef','steak','sirloin','ham','trout','tenderloin','tuna']
  Grains_bread=['pasta','rice','bread']
  oil=['butter','oil']
  Diary=['egg','cheese','yogurt','milk']
  Produce=['onion','garlic']
  fruits=['apple', 'apricot','banana','blackberry','cherry','grapefruit','mango','kiwi','orange','papaya','passion fruit'
  ,'peach','pear','pea','pineapple','plum','watermelon','raspberry','blueberry','grape']
  Vegetables=['asparagus','cabbage','avocado','bean','broccoli','corn','cucumber','carrot','eggplant','lemon','lettuce','mushroom','potatoe',
            'tomato','green pepper','asparagus','brussels sprout','parsley','oregano','thyme','rosemary','basil','lavender']
  All=fruits+Vegetables+Produce+Diary+oil+Grains_bread+Meat
  # Scrape Flipp
  import requests
  import pandas as pd


  BASE_URL = 'https://flipp.com'
  BACKEND_URL = 'https://backflipp.wishabi.com/flipp'
  SEARCH_URL = '%s/items/search' % BACKEND_URL
  ITEM_URL = '%s/items/' % BACKEND_URL

  # def scrape_item(item_id):
  #     return requests.get(
  #         "%s/%s" % (ITEM_URL, item_id,)
  #     ).json()

  def search(query, postal_code, locale):
    data = requests.get(
        SEARCH_URL,
        params = {
            'q': query,
            'postal_code': postal_code,
            'locale':locale
        }
    ).json()

    return [
  #             scrape_item(x.get('flyer_item_id'))

            x for x in data.get('items')
    ]

  # testing search function
  search_result = search('Milk','l6a3r7', 'en-ca')
  columns = list(search_result[0].keys())

  # Convert the product list to dataframe
  query_results=list()
  df=pd.DataFrame()
  for items in All:
  try:
    final_dict = {key:[] for key in columns}
    keys = final_dict.keys()
    query_results=search(items,'l6a3r7', 'en-ca')
    for q in query_results:
       for key in keys:
          final_dict[key].append(q[key])
    final_dict2=pd.DataFrame(final_dict)
    df=df.append(final_dict2)
    row=final_dict2.shape[0]
    df.index=range(df.shape[0])
    df.loc[df.shape[0]-row:,'product']=items
  except:
    continue

  df_sort=df.sort_values(['merchant_name'])
  mechant_name=df_sort.merchant_name.unique()
  df_sort["current_price"] = df_sort["current_price"].apply(pd.to_numeric)
  df_sort2=df_sort[df_sort.current_price>0]
  # Select the grocery sores from the merchant list

  food_store=[]
  for x in mechant_name:
  if 'food' in x.lower():
    food_store.append(x)
  if 'supermarket' in x.lower():
    food_store.append(x)
  if 'superstore' in x.lower():
    food_store.append(x)
  if 'mart' in x.lower():
    food_store.append(x)
  if 'fresh' in x.lower():
    food_store.append(x)
  if x in ['Walmart','Zehrs','Starsky','Sobeys','Price Chopper','No Frills','Longos','Metro','Loblaws','Healthy Planet','Giant Tiger']:
    food_store.append(x)

  df_sort_food=df_sort2[df_sort2.merchant_name.isin(food_store)]
  df_sort_food.index=range(df_sort_food.shape[0])
  df_sort_food2=df_sort_food



  # Image processing and create labels for images
  import io
  import os

  # Imports the Google Cloud client library
  from google.cloud import vision
  from google.cloud.vision import types

  # Instantiates a client
  client = vision.ImageAnnotatorClient()

  # The name of the image file to annotate

  import io
  import requests
  from PIL import Image
  import matplotlib.pyplot as plt  
  def Labeled_image(url):
  data = requests.get(url).content
  image = types.Image(content=data)
  img = Image.open(io.BytesIO(data))
  response = client.label_detection(image=image)
  labels = response.label_annotations
  Label_img=list()
  #   plt.imshow(img)
  #   plt.show()
  for label in labels:
    Label_img.append(label.description)
  return Label_img

  df_sort_food2['labels']=df_sort_food2['clean_image_url'].apply(lambda x:Labeled_image(str(x)))

  # Filter for relevant photos
  for i in df_sort_food2.index:
  df_sort_food2.loc[i,'Relevant']=pd.Series(df_sort_food2.loc[i,'labels']).isin(['Fruit','Fish','Meat','Vegetable','Ingredient','Dairy','Oil','Bread','Beef', 'Veal',  'Steak' ]).any()

  df_sort_food2=df_sort_food2[df_sort_food2.Relevant==True]

  # OCR flyers for weight analysis
  def detect_text_url(uri):
  try:
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
  #     print('Texts:')
    return texts[0].description
  except:
    return 'error'


  for i in df_sort_food2.index:
  if df_sort_food2.loc[i,"Relevant"]==True:
    df_sort_food2.loc[i,"ocr_name"]=detect_text_url(df_sort_food2.loc[i,"clean_image_url"])


  import re
  import ast
  # df_sort_food2['vagetables']=[any([i == 'Vegetable' for i in ast.literal_eval(x)]) for x in df_sort_food2.labels ]
  df_sort_food2['vagetables']=[any([i == 'Vegetable' for i in (x)]) for x in df_sort_food2.labels ]
  df_sort_food2['fruit']=[any([i == 'Fruit' for i in (x)]) for x in df_sort_food2.labels ]
  df_sort_food2['meat']=[any([i == 'Meat' for i in (x)]) for x in df_sort_food2.labels ]
  df_sort_food2['veal']=[any([i == 'Veal' for i in (x)]) for x in df_sort_food2.labels ]
  df_sort_food2['meat_chicken']=[any(re.findall('chicken',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['meat_steak']=[any(re.findall('steak',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['meat_ground']=[any(re.findall('ground',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['meat_tenderloin']=[any(re.findall('tenderloin',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['meat_serloin']=[any(re.findall('sirloin',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['rice']=[any(re.findall('rice',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['bread']=[any(re.findall('bread',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['milk']=[any(re.findall('milk',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['egg']=[any(re.findall('egg',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['yogurt']=[any(re.findall('yogurt',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['butter']=[any(re.findall('butter',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['oil']=[any(re.findall('oil',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['cheese']=[any(re.findall('cheese',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]
  df_sort_food2['pasta']=[any(re.findall('pasta',x, re.IGNORECASE))  for x in df_sort_food2.ocr_name ]

  df_sort_food2['All']= df_sort_food2[['milk','egg','butter','pasta','oil','cheese','vagetables','fruit','meat','veal','meat_chicken','meat_steak','meat_ground','meat_tenderloin','meat_serloin','rice','bread']].any(axis='columns')


  df_sort_food2=df_sort_food2[df_sort_food2['All']==True]
  # Extract weight numbers from ocr text
  import re
  import inflect
  p = inflect.engine()
  def Weight(name,product):
  pattern1 = re.compile(r'((?:[0-9]*\s*Pack)'
    r'|(?:[0-9.]+/*\s*/*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b))' , re.IGNORECASE)
  pattern2 = re.compile(r'('+str(p.plural(product))
                       +r'|'  +str(p.singular_noun(product))
                       +r'|' + str(product) +r')', re.IGNORECASE)

  pattern3 = re.compile(r'('+str(p.plural(product))
   +r'|' + str(product)+r'|((?:[0-9]*\s*Pack)'
    r'|(?:[0-9.]+/*\s*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b)))'
                       , re.IGNORECASE)
  return (re.findall(pattern1, name),re.findall(pattern2, name),re.findall(pattern3, name))

  import numpy as np
  def right_item(weight,product):
  r=weight
  D=np.array(r[2][:])
  try:

    o=0
    list2=list(map(lambda x:x.lower(),list(D[:,0])))

    list1=list(map(lambda x:x.lower(),list(r[0])))
    if p.plural(product.lower()) in list2:
      prod=p.plural(product).lower()
    if product.lower() in list2:
      prod=product.lower()

    try:
      x=list2.index(prod)
    except:
      x=0
      o=1
    if len(r[0])>1:
      while o==0 and x<=len(list2)-2:
        x+=1
        if list2[x] in list1:
          o=1

      if len(r[0])==1 or(o==0):
        x=0
      out=list2[x]

    if len(r[0])==1:
      out=list1[0]
    if len(r[0])==0:
      out='NaN'
  except:
    out='NaN'
  return out

  for i in df_sort_food2.index:
  list_weight1=Weight(df_sort_food2.ocr_name[i],df_sort_food2.loc[i,"product"])
  list_weight2=Weight(df_sort_food2.name[i],df_sort_food2.loc[i,"product"])
  product=df_sort_food2.loc[i,'product'].lower()
  if(list_weight1[0]>list_weight2[0]):
    list_weight=list_weight1
  else :
    list_weight=list_weight2
  if len(list_weight[0])==0 and len(list_weight[1])==0 :
    df_sort_food2.loc[i,'weight']='NaN'
  elif len(list_weight[0])==0 and len(list_weight[1])>=1 :
    df_sort_food2.loc[i,'weight']=1
  elif len(list_weight[0])>=1 and len(list_weight[1])==0 :
    df_sort_food2.loc[i,'weight']=right_item(list_weight,product)
  elif len(list_weight[0])>=1 and len(list_weight[1])>=1 :
    df_sort_food2.loc[i,'weight']=right_item(list_weight,product)  

  for i in df_sort_food2.index:
  df_sort_food2.loc[i,'weight_num']=0  
  df_sort_food2.loc[i,'price_100']=0  
  try:
    if df_sort_food2.loc[i,'weight']== 1:
      df_sort_food2.loc[i,'weight_num']=1  
    elif not any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'kg' :
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0])*1000
    elif any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'kg' :
      df_sort_food2.loc[i,'price_100']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-3])[0] )/10
    elif not any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-1].lower()=='g' and df_sort_food2.loc[i,'weight'][-2:].lower() != 'kg':  
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-1])[0] )
    elif df_sort_food2.loc[i,'weight'][-1].lower()=='l' and df_sort_food2.loc[i,'weight'][-2:].lower()=='ml':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )
    elif df_sort_food2.loc[i,'weight'][-1].lower()=='l' and df_sort_food2.loc[i,'weight'][-2:].lower()!='ml':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-1])[0] )*1000
    elif not any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'lb':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )*453.592
    elif any(re.findall('/', df_sort_food2.loc[i,'weight'])) and df_sort_food2.loc[i,'weight'][-2:].lower() == 'lb':
      df_sort_food2.loc[i,'price_100']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )/4.53592
    elif df_sort_food2.loc[i,'weight'][-2:].lower() == 'oz':
      df_sort_food2.loc[i,'weight2']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'][:-2])[0] )*28.3495
    elif df_sort_food2.loc[i,'weight'][-4:].lower() == 'pack':
      df_sort_food2.loc[i,'weight_num']=float(re.findall("[-+]?\d*\.\d+|\d+",df_sort_food2.loc[i,'weight'])[0] )  
  except:
    df_sort_food2.loc[i,'weight2']==0
  if df_sort_food2.loc[i,'price_100']==0:
    df_sort_food2.loc[i,'price_100']=df_sort_food2.loc[i,'current_price']/(df_sort_food2.loc[i,'weight2']/100)

  df_agg = df_sort_food2.groupby(['merchant_name','product'],observed=True)['merchant_name','product','current_price','weight2','weight_num','price_100','clean_image_url'].agg({'price_100':min})
  df_agg.columns = df_agg.columns.droplevel(0)
  df_agg = df_agg.reset_index()
  df_agg_price_sum=df_agg.groupby(['merchant_name']).agg({'current_price':sum}).rename(index=str, columns={"current_price": "Price_sum"})
  df_agg_price_sum = df_agg_price_sum.reset_index()
  df_agg_price_mean=df_agg[['product','price_100']].groupby(['product']).median().rename(index=str, columns={"price_100": "price_100_median"})
  df_agg_price_mean = df_agg_price_mean.reset_index()
  df_final=df_agg.merge(df_agg_price_sum,on='merchant_name')
  df_final=df_final.merge(df_agg_price_mean,on='product')
  df_final2=df_final[['product','merchant_name']].groupby(['merchant_name']).count()*100/len(df_sort_food2["product"].unique())
  df_final2 = df_final2.rename(columns={"product": "Percentage_list"}).reset_index()
  df_final3=df_final.merge(df_final2, on='merchant_name')
  df_final3=df_final3.sort_values('Percentage_list',ascending=False)
  return df_final3

     |████████████████████████████████| 419kB 4.5MB/s 


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

In [0]:
import pandas as pd
pd.set_option('max_colwidth', 200)

In [9]:
item_list=grocery_list()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

ValueError: ignored

In [0]:
  from google.colab import files
  from google.colab import drive
  item_list.to_csv('weekly_list.csv') 
  files.download('weekly_list.csv')
  